# Packages

In [248]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeClassifier, Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn import metrics 


# DATASET

## LOAD TRAIN

In [186]:
train =  pd.read_csv('MLUnige2023_subscriptions_train.csv', index_col='Id', na_values='na', header=0)

train = train.astype({'job':'category', 'marital':'category', 'education':'category', 'device':'category', 'outcome_old':'category', 'X1':'bool', 'X2':'bool', 'X3':'bool', 'X4':'float64', 'subscription':'category', 'day':'category', 'month':'category'})

train.day = pd.Categorical(train.day, ordered=True)

train.month = pd.Categorical(train.month, ordered=True)

train


,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,outcome_old,X1,X2,X3,X4,subscription
Id,,,,,,,,,,,,,,,,,
0,28,freelance,married,grad_school,smartphone,4,2,26.80,3,4,196,failure,False,False,True,0.072803,1
1,48,industrial_worker,married,university,smartphone,30,4,13.05,1,1,79,success,False,False,False,0.075454,1
2,27,teacher,married,university,smartphone,14,7,8.10,3,0,-1,NaN,False,True,True,0.068110,0
3,44,unemployed,divorced,university,smartphone,13,5,7.10,2,1,369,other,False,False,True,0.091942,0
4,29,manager,single,grad_school,smartphone,26,4,15.90,2,2,143,success,False,False,False,0.085922,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8947,54,industrial_worker,married,university,smartphone,16,7,7.30,1,0,-1,NaN,True,False,False,0.072803,0
8948,43,industrial_worker,married,university,smartphone,4,2,37.75,2,0,-1,NaN,False,False,True,0.081456,1
8949,27,manager,single,grad_school,NaN,5,6,29.00,3,0,-1,NaN,False,False,True,0.079186,1


In [187]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8952 entries, 0 to 8951
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               8952 non-null   int64   
 1   job               8891 non-null   category
 2   marital           8952 non-null   category
 3   education         8574 non-null   category
 4   device            6902 non-null   category
 5   day               8952 non-null   category
 6   month             8952 non-null   category
 7   time_spent        8952 non-null   float64 
 8   banner_views      8952 non-null   int64   
 9   banner_views_old  8952 non-null   int64   
 10  days_elapsed_old  8952 non-null   int64   
 11  outcome_old       2148 non-null   category
 12  X1                8952 non-null   bool    
 13  X2                8952 non-null   bool    
 14  X3                8952 non-null   bool    
 15  X4                8952 non-null   float64 
 16  subscription      8952 n

## MISSING TRAIN

In [188]:
missing_values = train.isnull().mean()

print(missing_values)

age                 0.000000
job                 0.006814
marital             0.000000
education           0.042225
device              0.228999
day                 0.000000
month               0.000000
time_spent          0.000000
banner_views        0.000000
banner_views_old    0.000000
days_elapsed_old    0.000000
outcome_old         0.760054
X1                  0.000000
X2                  0.000000
X3                  0.000000
X4                  0.000000
subscription        0.000000
dtype: float64


## LOAD TEST

In [189]:
test = pd.read_csv('MLUnige2023_subscriptions_test.csv', index_col='Id', na_values='na', header=0)

test = test.astype({'job':'category', 'marital':'category', 'education':'category', 'device':'category', 'outcome_old':'category', 'X1':'bool', 'X2':'bool', 'X3':'bool', 'X4':'float64', 'day':'category', 'month':'category'})

test.day = pd.Categorical(test.day, ordered=True)

test.month = pd.Categorical(test.month, ordered=True)

test

,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,outcome_old,X1,X2,X3,X4
Id,,,,,,,,,,,,,,,,
0,53,retired,married,university,NaN,17,6,5.25,1,0,-1,NaN,False,False,True,0.084570
1,61,manager,married,grad_school,smartphone,20,4,9.00,1,0,-1,NaN,False,False,False,0.075227
2,51,industrial_worker,married,university,NaN,4,7,9.65,1,0,-1,NaN,False,False,False,0.075781
3,34,manager,married,grad_school,NaN,28,5,13.45,2,0,-1,NaN,False,False,True,0.070043
4,30,manager,married,grad_school,smartphone,4,5,16.15,2,0,-1,NaN,False,False,False,0.171618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,47,industrial_worker,married,NaN,NaN,5,6,19.20,2,0,-1,NaN,False,False,True,0.104216
3833,30,teacher,single,university,smartphone,3,3,3.65,1,0,-1,NaN,True,False,False,0.077552
3834,55,teacher,married,university,NaN,26,6,5.15,5,0,-1,NaN,False,False,True,0.129156


In [190]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3837 entries, 0 to 3836
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               3837 non-null   int64   
 1   job               3816 non-null   category
 2   marital           3837 non-null   category
 3   education         3677 non-null   category
 4   device            2936 non-null   category
 5   day               3837 non-null   category
 6   month             3837 non-null   category
 7   time_spent        3837 non-null   float64 
 8   banner_views      3837 non-null   int64   
 9   banner_views_old  3837 non-null   int64   
 10  days_elapsed_old  3837 non-null   int64   
 11  outcome_old       900 non-null    category
 12  X1                3837 non-null   bool    
 13  X2                3837 non-null   bool    
 14  X3                3837 non-null   bool    
 15  X4                3837 non-null   float64 
dtypes: bool(3), category(7),

## MISSING TEST

In [191]:
missing_values = test.isnull().mean()

print(missing_values)

age                 0.000000
job                 0.005473
marital             0.000000
education           0.041699
device              0.234819
day                 0.000000
month               0.000000
time_spent          0.000000
banner_views        0.000000
banner_views_old    0.000000
days_elapsed_old    0.000000
outcome_old         0.765442
X1                  0.000000
X2                  0.000000
X3                  0.000000
X4                  0.000000
dtype: float64


## NA VALUES TRAIN

In [192]:
train['days_elapsed_old'] = train['days_elapsed_old'].replace(-1, np.nan)
train

,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,outcome_old,X1,X2,X3,X4,subscription
Id,,,,,,,,,,,,,,,,,
0,28,freelance,married,grad_school,smartphone,4,2,26.80,3,4,196.0,failure,False,False,True,0.072803,1
1,48,industrial_worker,married,university,smartphone,30,4,13.05,1,1,79.0,success,False,False,False,0.075454,1
2,27,teacher,married,university,smartphone,14,7,8.10,3,0,NaN,NaN,False,True,True,0.068110,0
3,44,unemployed,divorced,university,smartphone,13,5,7.10,2,1,369.0,other,False,False,True,0.091942,0
4,29,manager,single,grad_school,smartphone,26,4,15.90,2,2,143.0,success,False,False,False,0.085922,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8947,54,industrial_worker,married,university,smartphone,16,7,7.30,1,0,NaN,NaN,True,False,False,0.072803,0
8948,43,industrial_worker,married,university,smartphone,4,2,37.75,2,0,NaN,NaN,False,False,True,0.081456,1
8949,27,manager,single,grad_school,NaN,5,6,29.00,3,0,NaN,NaN,False,False,True,0.079186,1


In [193]:
train['X1'] = train['X1'].astype(int)
train['X2'] = train['X2'].astype(int)
train['X3'] = train['X3'].astype(int)

train = train.astype({'job':'category', 'marital':'category', 'education':'category', 'device':'category', 'X1':'category', 'X2':'category', 'X3':'category', 'X4':'float64', 'subscription':'category', 'day':'category', 'month':'category'})

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8952 entries, 0 to 8951
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               8952 non-null   int64   
 1   job               8891 non-null   category
 2   marital           8952 non-null   category
 3   education         8574 non-null   category
 4   device            6902 non-null   category
 5   day               8952 non-null   category
 6   month             8952 non-null   category
 7   time_spent        8952 non-null   float64 
 8   banner_views      8952 non-null   int64   
 9   banner_views_old  8952 non-null   int64   
 10  days_elapsed_old  2149 non-null   float64 
 11  outcome_old       2148 non-null   category
 12  X1                8952 non-null   category
 13  X2                8952 non-null   category
 14  X3                8952 non-null   category
 15  X4                8952 non-null   float64 
 16  subscription      8952 n

In [194]:
train

,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,outcome_old,X1,X2,X3,X4,subscription
Id,,,,,,,,,,,,,,,,,
0,28,freelance,married,grad_school,smartphone,4,2,26.80,3,4,196.0,failure,0,0,1,0.072803,1
1,48,industrial_worker,married,university,smartphone,30,4,13.05,1,1,79.0,success,0,0,0,0.075454,1
2,27,teacher,married,university,smartphone,14,7,8.10,3,0,NaN,NaN,0,1,1,0.068110,0
3,44,unemployed,divorced,university,smartphone,13,5,7.10,2,1,369.0,other,0,0,1,0.091942,0
4,29,manager,single,grad_school,smartphone,26,4,15.90,2,2,143.0,success,0,0,0,0.085922,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8947,54,industrial_worker,married,university,smartphone,16,7,7.30,1,0,NaN,NaN,1,0,0,0.072803,0
8948,43,industrial_worker,married,university,smartphone,4,2,37.75,2,0,NaN,NaN,0,0,1,0.081456,1
8949,27,manager,single,grad_school,NaN,5,6,29.00,3,0,NaN,NaN,0,0,1,0.079186,1


## MISSING TRAIN

In [195]:
missing_values = train.isnull().mean()
print(missing_values)

age                 0.000000
job                 0.006814
marital             0.000000
education           0.042225
device              0.228999
day                 0.000000
month               0.000000
time_spent          0.000000
banner_views        0.000000
banner_views_old    0.000000
days_elapsed_old    0.759942
outcome_old         0.760054
X1                  0.000000
X2                  0.000000
X3                  0.000000
X4                  0.000000
subscription        0.000000
dtype: float64


## DROP DAY_ELAPSED_OLD AND OUTCOME_OLD TRAIN

In [196]:
train = train.drop(columns=['days_elapsed_old', 'outcome_old'])

## MISSING TRAIN

In [197]:
missing_values = train.isnull().mean()
print(missing_values)

age                 0.000000
job                 0.006814
marital             0.000000
education           0.042225
device              0.228999
day                 0.000000
month               0.000000
time_spent          0.000000
banner_views        0.000000
banner_views_old    0.000000
X1                  0.000000
X2                  0.000000
X3                  0.000000
X4                  0.000000
subscription        0.000000
dtype: float64


## NA VALUES TEST

In [198]:
test['days_elapsed_old'] = test['days_elapsed_old'].replace(-1, np.nan)

In [199]:
test['X1'] = test['X1'].astype(int)
test['X2'] = test['X2'].astype(int)
test['X3'] = test['X3'].astype(int)

test = test.astype({'job':'category', 'marital':'category', 'education':'category', 'device':'category', 'X1':'category', 'X2':'category', 'X3':'category', 'X4':'float64', 'day':'category', 'month':'category'})

test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3837 entries, 0 to 3836
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               3837 non-null   int64   
 1   job               3816 non-null   category
 2   marital           3837 non-null   category
 3   education         3677 non-null   category
 4   device            2936 non-null   category
 5   day               3837 non-null   category
 6   month             3837 non-null   category
 7   time_spent        3837 non-null   float64 
 8   banner_views      3837 non-null   int64   
 9   banner_views_old  3837 non-null   int64   
 10  days_elapsed_old  902 non-null    float64 
 11  outcome_old       900 non-null    category
 12  X1                3837 non-null   category
 13  X2                3837 non-null   category
 14  X3                3837 non-null   category
 15  X4                3837 non-null   float64 
dtypes: category(10), float64

In [200]:
test

,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,outcome_old,X1,X2,X3,X4
Id,,,,,,,,,,,,,,,,
0,53,retired,married,university,NaN,17,6,5.25,1,0,NaN,NaN,0,0,1,0.084570
1,61,manager,married,grad_school,smartphone,20,4,9.00,1,0,NaN,NaN,0,0,0,0.075227
2,51,industrial_worker,married,university,NaN,4,7,9.65,1,0,NaN,NaN,0,0,0,0.075781
3,34,manager,married,grad_school,NaN,28,5,13.45,2,0,NaN,NaN,0,0,1,0.070043
4,30,manager,married,grad_school,smartphone,4,5,16.15,2,0,NaN,NaN,0,0,0,0.171618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,47,industrial_worker,married,NaN,NaN,5,6,19.20,2,0,NaN,NaN,0,0,1,0.104216
3833,30,teacher,single,university,smartphone,3,3,3.65,1,0,NaN,NaN,1,0,0,0.077552
3834,55,teacher,married,university,NaN,26,6,5.15,5,0,NaN,NaN,0,0,1,0.129156


## MISSING TEST

In [201]:
missing_values = test.isnull().mean()
print(missing_values)

age                 0.000000
job                 0.005473
marital             0.000000
education           0.041699
device              0.234819
day                 0.000000
month               0.000000
time_spent          0.000000
banner_views        0.000000
banner_views_old    0.000000
days_elapsed_old    0.764921
outcome_old         0.765442
X1                  0.000000
X2                  0.000000
X3                  0.000000
X4                  0.000000
dtype: float64


## DROP DAYS_ELAPSED_OLD AND OUTCOME_OLD TEST


In [202]:
test = test.drop(columns=['days_elapsed_old', 'outcome_old'])

## MISSING TEST

In [203]:
missing_values = test.isnull().mean()
print(missing_values)

age                 0.000000
job                 0.005473
marital             0.000000
education           0.041699
device              0.234819
day                 0.000000
month               0.000000
time_spent          0.000000
banner_views        0.000000
banner_views_old    0.000000
X1                  0.000000
X2                  0.000000
X3                  0.000000
X4                  0.000000
dtype: float64


## CATEGORICAL VS NUMERICAL COLUMNS

In [204]:
categorical_columns = train.select_dtypes(include=["category","bool", "object"])
categorical_columns_names = list(categorical_columns.columns)
numerical_columns = train.select_dtypes(exclude=["category","bool", "object"])
numerical_columns_names = list(numerical_columns)

print("Numerical:",numerical_columns_names, "\nCategory:",categorical_columns_names)

Numerical: ['age', 'time_spent', 'banner_views', 'banner_views_old', 'X4'] 
Category: ['job', 'marital', 'education', 'device', 'day', 'month', 'X1', 'X2', 'X3', 'subscription']


## REMOVING NA

In [205]:
train = train.dropna()
test = test.dropna()

In [130]:
train

,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,X1,X2,X3,X4,subscription
Id,,,,,,,,,,,,,,,
0,28,freelance,married,grad_school,smartphone,4,2,26.80,3,4,0,0,1,0.072803,1
1,48,industrial_worker,married,university,smartphone,30,4,13.05,1,1,0,0,0,0.075454,1
2,27,teacher,married,university,smartphone,14,7,8.10,3,0,0,1,1,0.068110,0
3,44,unemployed,divorced,university,smartphone,13,5,7.10,2,1,0,0,1,0.091942,0
4,29,manager,single,grad_school,smartphone,26,4,15.90,2,2,0,0,0,0.085922,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8946,32,unemployed,single,university,smartphone,2,8,23.80,1,2,0,0,0,0.148194,0
8947,54,industrial_worker,married,university,smartphone,16,7,7.30,1,0,1,0,0,0.072803,0
8948,43,industrial_worker,married,university,smartphone,4,2,37.75,2,0,0,0,1,0.081456,1


In [206]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6607 entries, 0 to 8951
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               6607 non-null   int64   
 1   job               6607 non-null   category
 2   marital           6607 non-null   category
 3   education         6607 non-null   category
 4   device            6607 non-null   category
 5   day               6607 non-null   category
 6   month             6607 non-null   category
 7   time_spent        6607 non-null   float64 
 8   banner_views      6607 non-null   int64   
 9   banner_views_old  6607 non-null   int64   
 10  X1                6607 non-null   category
 11  X2                6607 non-null   category
 12  X3                6607 non-null   category
 13  X4                6607 non-null   float64 
 14  subscription      6607 non-null   category
dtypes: category(10), float64(2), int64(3)
memory usage: 377.1 KB


In [207]:
missing_values = train.isnull().mean()
print(missing_values)

age                 0.0
job                 0.0
marital             0.0
education           0.0
device              0.0
day                 0.0
month               0.0
time_spent          0.0
banner_views        0.0
banner_views_old    0.0
X1                  0.0
X2                  0.0
X3                  0.0
X4                  0.0
subscription        0.0
dtype: float64


In [208]:
test

,age,job,marital,education,device,day,month,time_spent,banner_views,banner_views_old,X1,X2,X3,X4
Id,,,,,,,,,,,,,,
1,61,manager,married,grad_school,smartphone,20,4,9.00,1,0,0,0,0,0.075227
4,30,manager,married,grad_school,smartphone,4,5,16.15,2,0,0,0,0,0.171618
5,25,technology,single,grad_school,desktop,9,6,10.55,7,0,0,0,0,0.105760
6,32,freelance,married,university,smartphone,9,3,10.35,2,0,0,0,0,0.076517
7,41,unemployed,married,grad_school,smartphone,30,10,18.15,3,0,0,0,0,0.072803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,36,technology,single,university,smartphone,22,8,14.95,3,0,0,0,0,0.078832
3831,46,technology,married,grad_school,smartphone,20,11,26.05,1,1,0,0,0,0.072803
3833,30,teacher,single,university,smartphone,3,3,3.65,1,0,1,0,0,0.077552


In [209]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2823 entries, 1 to 3836
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               2823 non-null   int64   
 1   job               2823 non-null   category
 2   marital           2823 non-null   category
 3   education         2823 non-null   category
 4   device            2823 non-null   category
 5   day               2823 non-null   category
 6   month             2823 non-null   category
 7   time_spent        2823 non-null   float64 
 8   banner_views      2823 non-null   int64   
 9   banner_views_old  2823 non-null   int64   
 10  X1                2823 non-null   category
 11  X2                2823 non-null   category
 12  X3                2823 non-null   category
 13  X4                2823 non-null   float64 
dtypes: category(9), float64(2), int64(3)
memory usage: 159.9 KB


In [210]:
missing_values = test.isnull().mean()
print(missing_values)

age                 0.0
job                 0.0
marital             0.0
education           0.0
device              0.0
day                 0.0
month               0.0
time_spent          0.0
banner_views        0.0
banner_views_old    0.0
X1                  0.0
X2                  0.0
X3                  0.0
X4                  0.0
dtype: float64


# SCALING

In [211]:
train_num = train[numerical_columns_names]
train_cat = train[categorical_columns_names]

test_num = test[numerical_columns_names]
test_cat = test[categorical_columns_names[:-1]]

# create a StandardScaler object
scaler = StandardScaler()

# fit and transform the scaler on the numerical variables
scaler.fit(train_num)
train_num = pd.DataFrame(scaler.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num = pd.DataFrame(scaler.transform(test_num),columns=test_num.columns, index=test_num.index)

# merge again 
train = pd.merge(train_num, train_cat, how="left", on=["Id"])
test = pd.merge(test_num, test_cat, how="left", on=["Id"])

In [212]:
train

,age,time_spent,banner_views,banner_views_old,X4,job,marital,education,device,day,month,X1,X2,X3,subscription
Id,,,,,,,,,,,,,,,
0,-1.099928,0.572138,0.203390,1.276111,-0.450964,freelance,married,grad_school,smartphone,4,2,0,0,1,1
1,0.555193,-0.254284,-0.571357,0.006597,-0.368693,industrial_worker,married,university,smartphone,30,4,0,0,0,1
2,-1.182684,-0.551796,0.203390,-0.416574,-0.596628,teacher,married,university,smartphone,14,7,0,1,1,0
3,0.224169,-0.611899,-0.183983,0.006597,0.142962,unemployed,divorced,university,smartphone,13,5,0,0,1,0
4,-1.017172,-0.082989,-0.183983,0.429768,-0.043837,manager,single,grad_school,smartphone,26,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8946,-0.768904,0.391828,-0.571357,0.429768,1.888677,unemployed,single,university,smartphone,2,8,0,0,0,0
8947,1.051730,-0.599878,-0.571357,-0.416574,-0.450964,industrial_worker,married,university,smartphone,16,7,1,0,0,0
8948,0.141413,1.230271,-0.183983,-0.416574,-0.182457,industrial_worker,married,university,smartphone,4,2,0,0,1,1


In [213]:
test

,age,time_spent,banner_views,banner_views_old,X4,job,marital,education,device,day,month,X1,X2,X3
Id,,,,,,,,,,,,,,
1,1.631022,-0.497703,-0.571357,-0.416574,-0.375737,manager,married,grad_school,smartphone,20,4,0,0,0
4,-0.934416,-0.067963,-0.183983,-0.416574,2.615589,manager,married,grad_school,smartphone,4,5,0,0,0
5,-1.348196,-0.404542,1.752884,-0.416574,0.571784,technology,single,grad_school,desktop,9,6,0,0,0
6,-0.768904,-0.416563,-0.183983,-0.416574,-0.335729,freelance,married,university,smartphone,9,3,0,0,0
7,-0.024099,0.052244,0.203390,-0.416574,-0.450964,unemployed,married,grad_school,smartphone,30,10,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,-0.437879,-0.140087,0.203390,-0.416574,-0.263883,technology,single,university,smartphone,22,8,0,0,0
3831,0.389681,0.527061,-0.571357,0.006597,-0.450964,technology,married,grad_school,smartphone,20,11,0,0,0
3833,-0.934416,-0.819256,-0.571357,-0.416574,-0.303609,teacher,single,university,smartphone,3,3,1,0,0


## HOT-ENCODING

In [225]:
train_hot = pd.get_dummies(train, columns=['job','marital','education','device','day','month'])
test_hot = pd.get_dummies(test, columns=['job','marital','education','device','day','month'])

### TRAIN

In [226]:
train_hot

,age,time_spent,banner_views,banner_views_old,X4,X1,X2,X3,subscription,job_entrepreneur,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Id,,,,,,,,,,,,,,,,,,,,,
0,-1.099928,0.572138,0.203390,1.276111,-0.450964,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.555193,-0.254284,-0.571357,0.006597,-0.368693,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,-1.182684,-0.551796,0.203390,-0.416574,-0.596628,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.224169,-0.611899,-0.183983,0.006597,0.142962,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,-1.017172,-0.082989,-0.183983,0.429768,-0.043837,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8946,-0.768904,0.391828,-0.571357,0.429768,1.888677,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8947,1.051730,-0.599878,-0.571357,-0.416574,-0.450964,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8948,0.141413,1.230271,-0.183983,-0.416574,-0.182457,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_hot.info()

### TEST

In [ ]:
test_hot

In [ ]:
test_hot.info()

## DROPPING SUBSCRIPTION FOR PREDICTIONS

In [244]:
train_no_subscription = train_hot.drop('subscription', axis=1)
train_only_subscription = train_hot['subscription']

# TRAIN, TEST AND VALIDATION

In [257]:
valid_size = 0.3
train_size = 1-valid_size

# Split the data into training and test sets
X_train, X_valid, Y_train, Y_valid = train_test_split(train_no_subscription, train_only_subscription, test_size=valid_size)

In [ ]:
X_train

In [251]:
Cond1 = X_train.shape[0] == Y_train.shape[0]

Cond2 = X_valid.shape[0] == Y_valid.shape[0]

print("\nSame Size ?","\n\nFor Training:",Cond1,", For Validation:", Cond2)

PropTV = Y_valid.shape[0]/Y_train.shape[0]*100

print("\nProportions between Training and Validation:", round(PropTV,2),"%")


Same Size ? 

For Training: True , For Validation: True

Proportions between Training and Validation: 42.88 %


# PREDICTIVE MODELS

## DECISION TREES

In [249]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,Y_train)

#Predict the response for test dataset
TREES_PRED = clf.predict(X_valid)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_valid, TREES_PRED))

Accuracy: 0.7710934544078698
